В среде **Google Colab** может быть доступен графический процессор **NVIDIA Tesla T4**, который подходит для выполнения параллельных вычислений с использованием технологии **CUDA**, включая реализацию алгоритмов сортировки на GPU (например, через CUDA C++, Numba или PyCUDA).

Основные характеристики **NVIDIA Tesla T4**:

* количество CUDA-ядер: **2560**;
* объём видеопамяти: **16 ГБ GDDR6**;
* производительность в вычислениях с плавающей точкой (FP32): около **8,1 TFLOPS**.

Данный графический процессор ориентирован на задачи параллельной обработки данных, машинного обучения и высокопроизводительных вычислений. Его вычислительных ресурсов достаточно для эффективной обработки массивов размером **10 000–100 000 элементов** и более, в том числе при реализации параллельных алгоритмов сортировки на GPU.



In [1]:
!nvidia-smi

Mon Dec 29 19:04:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!apt update
!apt install -y nvidia-cuda-toolkit

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,225 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,572 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/univer

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [16]:
%%writefile task_4_cuda_merge_sort.cu

#include <iostream>        // Стандартная библиотека ввода-вывода
#include <vector>          // Контейнер vector
#include <algorithm>       // Стандартные алгоритмы (min, max)
#include <cuda_runtime.h>  // CUDA Runtime API
#include <random>          // Генерация случайных чисел
#include <chrono>          // Измерение времени выполнения
#include <omp.h>           // OpenMP для параллельных вычислений на CPU

using namespace std;       // Использование стандартного пространства имён

// ======================================================
// CUDA kernel: сортировка подмассива внутри одного CUDA-блока
// Используется shared memory для ускорения доступа
// ======================================================
__global__ void sortBlocksSharedKernel(int* data, int blockSize, int n) {

    extern __shared__ int s[];                 // Динамическая shared memory

    int tid = threadIdx.x;                     // Локальный индекс потока в блоке
    int gid = blockIdx.x * blockSize + tid;    // Глобальный индекс элемента

    if (gid < n)
        s[tid] = data[gid];                    // Копирование данных в shared memory
    else
        s[tid] = INT_MAX;                      // Заполнение фиктивным значением

    __syncthreads();                           // Синхронизация потоков блока

    for (int i = 1; i < blockSize; i++) {      // Сортировка вставками
        int key = s[i];                        // Текущий элемент
        int j = i - 1;                         // Индекс предыдущего элемента

        while (j >= 0 && s[j] > key) {          // Сдвиг элементов вправо
            s[j + 1] = s[j];
            j--;
        }

        s[j + 1] = key;                        // Вставка элемента
        __syncthreads();                       // Синхронизация потоков
    }

    if (gid < n)
        data[gid] = s[tid];                    // Запись результата в глобальную память
}

// ======================================================
// Слияние двух отсортированных подмассивов (CPU)
// ======================================================
void merge(int* arr, int* temp, int left, int mid, int right) {

    int i = left;      // Индекс левого подмассива
    int j = mid;       // Индекс правого подмассива
    int k = left;      // Индекс временного массива

    while (i < mid && j < right)
        temp[k++] = (arr[i] <= arr[j]) ? arr[i++] : arr[j++]; // Слияние

    while (i < mid) temp[k++] = arr[i++];   // Копирование остатка слева
    while (j < right) temp[k++] = arr[j++]; // Копирование остатка справа
}

// ======================================================
// Параллельная сортировка слиянием с использованием OpenMP
// ======================================================
void parallelMergeSortOMP(int* arr, int n) {

    int* temp = new int[n];                   // Временный массив

    for (int width = 1; width < n; width *= 2) {

        #pragma omp parallel for schedule(static)
        for (int i = 0; i < n; i += 2 * width) {
            int left  = i;                    // Левая граница
            int mid   = min(i + width, n);    // Середина
            int right = min(i + 2 * width, n);// Правая граница
            merge(arr, temp, left, mid, right);// Слияние
        }

        #pragma omp parallel for schedule(static)
        for (int i = 0; i < n; i++)
            arr[i] = temp[i];                 // Копирование результата
    }

    delete[] temp;                            // Освобождение памяти
}

// ======================================================
// Проверка корректности сортировки
// ======================================================
bool isSorted(const vector<int>& v) {

    for (size_t i = 1; i < v.size(); i++)
        if (v[i - 1] > v[i]) return false;    // Нарушение порядка

    return true;                              // Массив отсортирован
}

// ======================================================
// Главная функция
// ======================================================
int main() {

    vector<int> sizes = {100, 1000, 10000, 100000, 1000000}; // Размеры массивов

    random_device rd;                         // Источник энтропии
    mt19937 gen(rd());                        // Генератор Mersenne Twister
    uniform_int_distribution<int> dist(0, 1000000); // Диапазон чисел

    for (int n : sizes) {

        vector<int> h(n);                     // Массив на CPU
        for (int& x : h) x = dist(gen);       // Заполнение случайными числами

        cout << "========== Размер массива: " << n << " ==========\n";
        cout << "До сортировки (первые 10): ";
        for (int i = 0; i < min(10, n); i++) cout << h[i] << " ";

        cout << "\nДо сортировки (последние 10): ";
        for (int i = max(0, n - 10); i < n; i++) cout << h[i] << " ";
        cout << "\n";

        int* d;                               // Указатель на память GPU
        cudaMalloc(&d, n * sizeof(int));      // Выделение памяти на GPU
        cudaMemcpy(d, h.data(), n * sizeof(int),
                   cudaMemcpyHostToDevice);   // Копирование на GPU

        int blockSize = 1024;                 // Размер CUDA-блока
        int blocks = (n + blockSize - 1) / blockSize; // Количество блоков

        auto start = chrono::high_resolution_clock::now(); // Старт таймера

        sortBlocksSharedKernel<<<blocks, blockSize,
            blockSize * sizeof(int)>>>(d, blockSize, n);   // CUDA сортировка

        cudaDeviceSynchronize();              // Ожидание GPU

        cudaMemcpy(h.data(), d, n * sizeof(int),
                   cudaMemcpyDeviceToHost);   // Копирование обратно на CPU

        parallelMergeSortOMP(h.data(), n);    // Параллельное слияние (CPU)

        auto end = chrono::high_resolution_clock::now();   // Конец таймера
        chrono::duration<double, milli> elapsed = end - start;

        cout << "После сортировки (первые 10): ";
        for (int i = 0; i < min(10, n); i++) cout << h[i] << " ";

        cout << "\nПосле сортировки (последние 10): ";
        for (int i = max(0, n - 10); i < n; i++) cout << h[i] << " ";
        cout << "\n";

        cout << "Время сортировки: " << elapsed.count() << " мс\n";
        cout << (isSorted(h)
            ? "Массив отсортирован корректно\n\n"
            : "ОШИБКА СОРТИРОВКИ\n\n");

        cudaFree(d);                          // Освобождение памяти GPU
    }

    return 0;                                 // Завершение программы
}

Overwriting task_4_cuda_merge_sort.cu


In [17]:
!nvcc -Xcompiler -fopenmp task_4_cuda_merge_sort.cu -o task_4_cuda_merge_sort
!./task_4_cuda_merge_sort

========== Размер массива: 100 ==========
До сортировки (первые 10): 67850 41020 666177 230514 146459 844468 775607 576110 874024 489892 
До сортировки (последние 10): 431601 858818 776168 837216 357313 267208 364957 538709 929469 620910 
После сортировки (первые 10): 2477 5378 7879 35652 41020 45007 52714 55296 67850 70379 
После сортировки (последние 10): 882066 904206 908557 908802 921783 929469 938718 978752 983004 997850 
Время сортировки: 66.7938 мс
Массив отсортирован корректно

========== Размер массива: 1000 ==========
До сортировки (первые 10): 187336 459802 734602 769211 34689 661794 787790 688598 24524 477473 
До сортировки (последние 10): 18144 44405 440026 151917 118927 139662 880089 952348 956669 332030 
После сортировки (первые 10): 71 212 244 1570 1939 2702 2889 4177 4304 4551 
После сортировки (последние 10): 988549 990149 991124 992353 993921 995093 996417 996424 997677 998191 
Время сортировки: 16.4127 мс
Массив отсортирован корректно

========== Размер массива: 100